# CISC451/893 Competition 1

**TEAM:**<br>
Ryan Elliott - 05999293<br>
Sizhe Guan 20090459<br>
Vishal Chotara 20307118
## Question 1<br>
### Accessibility of Kingston transit
City of Kingston has an extensive transit network. The city maintains 23 routes helping connect the east and west urban areas of the city. The urban regions of the city are well serviced by the transit network. The data environment is prepared in this cell. City of Kingston data is fetched using the API. The data evaluated is all the table of all transit stops, table of all driveways (commericial and residential), and table of active busroutes in Kingston. The effectiveness of a transit network is highly correlated to the systems accessibility. Accessibility can mean different things depending on the study. This analysis will define accessbility to be a walking distance of 200m. We will assume that if a bus stop is within 200m of a driveway/address then the stop will be accessible.

In [ ]:
!!pip install -r requirements.txt

In [ ]:
# The data environment is prepared in this cell. City of Kingston data is fetched
# using the API

import requests
import pandas as pd
import numpy as np
import json
stop_api_url = "https://opendatakingston.cityofkingston.ca/api/records/1.0/search/?dataset=transit-gtfs-stops&q=&rows=799"
driveway_api_url = "https://opendatakingston.cityofkingston.ca/api/records/1.0/search/?dataset=driveways&q=&rows=-1&facet=material"
busroute_api_url = 'https://opendatakingston.cityofkingston.ca/api/records/1.0/search/?dataset=transit-gtfs-routes&q=&rows=-1&facet=route_short_name&exclude.route_long_name=999+Maintenance+-+Out+of+Service'
stop_response = requests.get(stop_api_url)
driveway_response = requests.get(driveway_api_url)
busroute_response = requests.get(busroute_api_url)

In [ ]:
# Data must be unpacked using json() method.
x = stop_response.json()
x_driveway = driveway_response.json()
x_busroute = busroute_response.json()

# The json() frames are then turned into Pandas dataframes.
transit_stops = pd.json_normalize(x,'records')
driveways = pd.json_normalize(x_driveway,'records')
busroute = pd.json_normalize(x_busroute,'records')

The city actively maintains and updates the opendata the databases. Sometimes the new records do not have GPS coordinates. The records without GPS coordinates are removed from analysis.

In [ ]:
# drops data if there is no GPS coordinates.
driveways = driveways.dropna(subset = ['fields.geo_point_2d'],how = 'all')

GPS coordinates are points on a sphere. Measuring the distance between poinst cannot be completed in cartesian space. The points must be projected onto a sphere to measure the arc length between the points. The haversin function is a common approximation used to measure the poits between two GPS coordinates.

In [ ]:
import math

def haversine(coord1,coord2):
    # Takes two coordinate vectors coord1 and coord2 and 
    # returns the distance between two points in metres
    
    # Radius of the Earth at Kingston. 
    # Assuming mean elevation of 93m above sea level.
    R = 6367869
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
    return 2*R*math.atan2(math.sqrt(a),math.sqrt(1-a))

In [ ]:
transit_stops['bus_routes'] = pd.Series()

c:\users\ryan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


Next we evaluate the distance of the driveways to all the bus stops. An array is created of the distance of each driveway to each bus stop.

In [ ]:
transit_lat = transit_stops['fields.stop_coordinates'].explode().to_numpy()[::2]
transit_long = transit_stops['fields.stop_coordinates'].explode().to_numpy()[1::2]
transit_coords = np.column_stack((transit_lat,transit_long))

driveway_lat = driveways['fields.geo_point_2d'].explode().to_numpy()[::2]
driveway_long =  driveways['fields.geo_point_2d'].explode().to_numpy()[1::2]

driveway_coords = np.column_stack((driveway_lat,driveway_long))

In [ ]:
stop_distance = np.ndarray(shape = [driveway_lat.size,transit_lat.size])
k = 0
l = 0

for i in driveway_coords:
    l = 0
    for j in transit_coords:
        stop_distance[k,l] = haversine(i,j)
        l = l+1
    k = k+1

In [ ]:
closest_stop = stop_distance.min(axis = 1)
print('median distance to a stop: ',round(np.median(closest_stop)),'m')
print('average distance to a stop: ',round(closest_stop.mean()),'m')

median distance to a stop:  199 m
average distance to a stop:  1287 m


Half of all the driveways in Kingston are within 200m of a bus stop. The average distance to a bus stop is heavily skewed by driveways that are not in the urban part of the city of Kingston.<br><br>
We may also evaluate which stops in the network are walkable. We find that 706 of 799 stops are within 200m 

In [ ]:
stop_density = stop_distance < 200
walkable_stop = stop_density.sum(axis=0)>0
print(walkable_stop.sum())

706


The next section of code is trying to determine which routes service which stops and can be used to determine which stops serve as potential transfers between bus routes. A stop is said to be part of a bus route if the haversine formula measures a distance of \<200m of the shaded route to the stop. The routes are stored in a list for that record in the data frame.

In [ ]:
# The calculation needs to be complete for all bus stops. Stops need to be recorded in the dataframe. TBD.
k = 0
for s in transit_stops['fields.stop_coordinates']:
    stop_test = np.array(s)
    route_list = list()
    for route, ind in enumerate(busroute['fields.shape.coordinates']):
        p = 0
        route_test = np.array(busroute['fields.shape.coordinates'][route][0])
        for ind in route_test:
            test_param = haversine(np.flip(route_test[p]),stop_test)
            if test_param<150:
                # Append bus route to list.
                route_list.append(busroute['fields.route_short_name'][route])
                # Loop breaks if stop identified as being within 100m of bus route.
                break


            p = p+1

    transit_stops['bus_routes'][k] = route_list
    # print(transit_stops['fields.stop_name'][k]," ",transit_stops['bus_routes'][k])
    k = k+1



c:\users\ryan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\ryan\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


The number of routes each stop services can be seen accross the network using a histogram. There are 38 stops which were not assigned to a route. This is likely due to the distcrete spacing of the GPS shape coordinates in the bus route feature. These can be binned with the stops that service a single route. What is pretty interesting is that there are 19 bus stops that have 10 bus routes within 200m of the stop.

In [ ]:
print(np.histogram(transit_stops['bus_routes'].str.len()))

(array([ 38, 461, 113, 109,  22,  17,   7,   4,   9,  19], dtype=int64), array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]))


In [ ]:
print(np.histogram(transit_stops['bus_routes'].str.len(),bins=[0,2,3,4,5,6,7,8,9,10]))

(array([499, 113, 109,  22,  17,   7,   4,   9,  19], dtype=int64), array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10]))


In [ ]:
print(round(100*499/799),'% of the transit network does not have a transfer.')

62 % of the transit network does not have a transfer.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ba3a38b4-573b-4649-8117-71de8550c212' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>